In [ ]:
# here we run auto-ARIMA which auto selects values of p and q that best match the autoregressive (AR) and moving average (MA) orders.

# then we do validation checks to make sure the auto-ARIMA model isnt giving us bs. 

# Partial Autocorrelation Function (PACF): Helps you identify the AR (p) order. 
# peaks outside the  bounds in the PACF plot suggest the lag order for the AR term.

# Autocorrelation Function (ACF):  identify the MA (q) order. 
#  lags in the ACF plot suggest the lag order for the MA term.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy.stats as stats
import numpy as np
from statsmodels.tsa.stattools import adfuller  


# load data and convert to time series -- both isem_original and isem_diff_1 meet stationarity assumptions so it is safe to use either.
df = pd.read_csv('data/arima/isem_original.csv')
df['StartDateTime'] = pd.to_datetime(df['StartDateTime'])
df.set_index('StartDateTime', inplace=True)

# test loading using dis 
# print(df.head())


### fitting a auto-arima model

In [ ]:
# FITTING THE AUTO-ARIMA MODEL!
# note: we use d = 0 here as we already figured that isem_original follows stationarity

# Split data into train and test
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Fit the model on training data
model = pm.auto_arima(train['ISEM DA Price'],
    start_p=0, max_p=5,
    start_q=0, max_q=5,
    start_d=0, max_d=2,  # Try d values from 0 to 2
    test='adf',          # Use ADF test to determine d
    seasonal=False,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True,
    information_criterion='aic')  # You can also use 'bic' or 'aicc'

print("\nBest ARIMA order:", model.order)
print(model.summary())

# Make predictions
predictions = model.predict(n_periods=len(test))

# Check for NaNs and infinite values in predictions
print("Number of NaNs in predictions:", np.isnan(predictions).sum())
print("Number of infinite values in predictions:", np.isinf(predictions).sum())

# Create a Series with the correct index
predictions = pd.Series(predictions, index=test.index)

# Plot results
plt.figure(figsize=(12,6))
plt.plot(train.index, train['ISEM DA Price'], label='Training Data')
plt.plot(test.index, test['ISEM DA Price'], label='Actual Test Data')
plt.plot(test.index, predictions, label='Predictions', color='red')
plt.title('Auto ARIMA Model: Actual vs Predicted')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


### validation

In [ ]:
# ================== VALIDATION SECTION ==================

print("Number of NaNs in data:", df['ISEM DA Price'].isna().sum())
print("Number of infinite values in data:", np.isinf(df['ISEM DA Price']).sum())
df = df[np.isfinite(df['ISEM DA Price'])]

# 1. Residual Analysis
residuals = test['ISEM DA Price'] - predictions

# Plot ACF and PACF of residuals
plt.figure(figsize=(12, 8))
plt.subplot(211)
plot_acf(residuals, ax=plt.gca(), title='ACF of Residuals')
plt.subplot(212)
plot_pacf(residuals, ax=plt.gca(), title='PACF of Residuals')
plt.tight_layout()
plt.show()

# 2. Stationarity Test on Residuals
adf_test = adfuller(residuals)  # Use adfuller from statsmodels.tsa.stattools
print("\nADF Test Results for Residuals:")
print(f'ADF Statistic: {adf_test[0]}')
print(f'p-value: {adf_test[1]}')
print('Critical values:')
for key, value in adf_test[4].items():
    print(f'\t{key}: {value}')

# 3. Forecast Accuracy Metrics
rmse = np.sqrt(mean_squared_error(test['ISEM DA Price'], predictions))
mae = mean_absolute_error(test['ISEM DA Price'], predictions)
mape = np.mean(np.abs((test['ISEM DA Price'] - predictions) / test['ISEM DA Price'])) * 100

print("\nForecast Accuracy Metrics:")
print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MAPE: {mape:.2f}%')

# Optional: Q-Q plot for residuals normality check
plt.figure(figsize=(10,6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q plot of residuals")
plt.show()

# Optional: Histogram of residuals
plt.figure(figsize=(10,6))
plt.hist(residuals, bins=50, density=True)
plt.title("Histogram of Residuals")
plt.xlabel("Residual Value")
plt.ylabel("Frequency")
plt.show()

#### ARIMA model evaluation summary

After running the above code with an *80% training – 20% test split* and validating the results, we can conclude the following:

- The **residuals are stationary** (ADF test statistic is very negative, p-value ≈ 0), meaning captures trend wel  and is still stationary
- The **ACF/PACF of residuals** show no significant spikes, meaning no obvious leftover patterns   - a good sign. 
- **RMSE = 40, MAE = 31**, mean the forecast still has some error. On average, MAE being 31 means predictions are off by **31 euros**, so while the model is reasonable, it's not perfect 
 

all in all, this confirms out goal to build a bunch of forecasting models that are just parts of a trader’s toolkit that all work decently, giving flexibility across different market conditions